## Transform a stock option input file and load it into BigQuery
- Calcualte equilibrium stock price where call and put implied volatility are equal
- Calculate moneyness for all strikes within +- 2 standard deviations of equilibrium price
- Calculate implied volatility of all strikes filtered above
- generate json formatted output
- write output file to bigquery

<b>Data Source:</b> https:\\historicaloptiondata.com

### Destination Table Field Defiinitions
- quote_date - market closing date for which the data was captured
- expiry_date - date on which the options expiry
- days_to_expiry _ calendar days between quote_date and expiry_date not including the quote_date
- underlying_price - the price of the underlying asset
- atm_price - adjusted underlying price such that the implied volatility of calls an puts are the same
- atm_iv - the implied volatility for both calls and puts using the atm_price as the current price
- strike_prices
 - strike_price - price at which an option can be excercised
 - call_bid - bid price for the call option 
 - call_ask - ask price for the call option
 - call_volume -  number of call contracts traded
 - call_open_iterest - number of open call contracts
 - call_moneyness - probability that the call option will close in the money on the expiry date (based on atm_implied_volatility)
 - call_iv - the implied volatiity of the call option using the midpoint between the call bid and call ask price
 - put_bid - bid price for the put option 
 - put_ask - ask price for the put option
 - put_volume -  number of put contracts traded
 - put_open_iterest - number of open put contracts
 - put_moneyness - probability that the put option will close in the money on the expiry date (base on atm_implied_volatiity)
 - put_iv - the implied volatiity of the put option using the midpoint between the put bid and put ask price
- sampling_key - a random number between 0 and 1. Facilitates repeatable data sampling without the need for a hash key 


## Codebase

In [61]:
#declare dependencies and constants
import pandas as pd
import datetime
import math
import mibian
import scipy
import json
import random

In [78]:
def create_option_bq_table():
    from google.cloud import bigquery
    client = bigquery.Client()
    dataset_ref = client.dataset('stock_options')

    schema = [
        bigquery.SchemaField("quote_date", "DATE", mode="REQUIRED"),
        bigquery.SchemaField("expiry_date", "DATE", mode="REQUIRED"),
        bigquery.SchemaField("days_to_expiry", "INT64", mode="REQUIRED"),
        bigquery.SchemaField("underlying_price", "NUMERIC", mode="REQUIRED"),
        bigquery.SchemaField("atm_price", "NUMERIC", mode="REQUIRED"),
        bigquery.SchemaField("atm_iv", "NUMERIC", mode="REQUIRED"),
        bigquery.SchemaField(
            "strike_prices",
            "RECORD",
            mode="REPEATED",
            fields=[
                bigquery.SchemaField("strike_price", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("call_bid", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("call_ask", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("call_volume", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("call_open_interest", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("call_moneyness", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("call_iv", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("put_bid", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("put_ask", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("put_volume", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("put_open_interest", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("put_moneyness", "NUMERIC", mode="REQUIRED"),
                bigquery.SchemaField("put_iv", "NUMERIC", mode="REQUIRED"),
            ],
        ),
        bigquery.SchemaField("sampling_key", "NUMERIC", mode="REQUIRED")
    ]
    table_ref = dataset_ref.table("SPY")
    table = bigquery.Table(table_ref, schema=schema)
    table = client.create_table(table)  # API request

    print("Created table {}".format(table.full_table_id))

In [3]:
def get_interest_rate(quote_date):
    "return the fed funds rate that was in effect on the supplied quote date"
    df_fedfunds = pd.read_csv('gs://expiry-week-data/options/FEDFUNDS.csv', parse_dates=['DATE'])
    df_fedfunds = df_fedfunds[df_fedfunds['DATE'] <= quote_date]
    target_index = df_fedfunds['DATE'].idxmax()
    return df_fedfunds.loc[target_index]['FEDFUNDS']


In [159]:
def calc_call_iv(stock_price, strike_price, interest_rate, days_to_expiry, call_price):
    """
    calculate the implied volatility of a call option
    - return annualized implied volatility as a decimal value
    """
    bs = mibian.BS([stock_price, strike_price, interest_rate, days_to_expiry], callPrice=call_price)
    return round(bs.impliedVolatility / 100, 4)

In [160]:
def calc_put_iv(stock_price, strike_price, interest_rate, days_to_expiry, put_price):
    """
    calculate the implied volatility of a put option
    - return annualized implied volatility as a decimal value
    """
    bs = mibian.BS([stock_price, strike_price, interest_rate, days_to_expiry], putPrice=put_price)
    return round(bs.impliedVolatility / 100, 4)

In [6]:
def center_underlying_price(stock_price, strike_price, interest_rate, days_to_expiry, call_price, put_price):
    """
    calculate equilibrium implied volatility and the adjusted underlying price at which it is acheived
    this is the underlying price at which both call and put implied volatility are equal
    note: interest_rate needs be passed as a percent (e.g 5 = 5%)
    """
    #get starting call and put implied volatility adjusted for time to expiry
    call_iv = calc_call_iv(stock_price, strike_price, interest_rate, days_to_expiry, call_price)       
    put_iv = calc_put_iv(stock_price, strike_price, interest_rate, days_to_expiry, put_price)
    
    #calculate implied volatility difference adjusted for time to expiry
    iv_diff = abs(put_iv - call_iv) * math.sqrt(days_to_expiry / 365) 

    adj_lower = stock_price  * math.exp(-iv_diff) 
    adj_upper = stock_price  * math.exp(iv_diff)
    if put_iv > call_iv:
        #stock price is above equilibrium price
        adj_stock_price = (stock_price + adj_lower) / 2
    else:
        #stock price is below equilibrium price
        adj_stock_price = (stock_price + adj_upper) / 2

    adj_stock_price = stock_price 
    for i in range(100):
        call_iv = calc_call_iv(adj_stock_price, strike_price, interest_rate, days_to_expiry, call_price)
        put_iv = calc_put_iv(adj_stock_price, strike_price, interest_rate, days_to_expiry, put_price)
        iv_diff = abs(put_iv - call_iv)

        #at point of convergence call and put iv will be the same, so return either 1
        if iv_diff <= .0001:
            return adj_stock_price, call_iv 
        
        if put_iv > call_iv:
            #adjusted stock price is to high
            adj_upper = adj_stock_price
        else:
            #adjusted stock price is to low
            adj_lower = adj_stock_price
       
        adj_stock_price = (adj_lower + adj_upper) / 2

    #throw error if convergence was not achieved
    raise Exception("Put and Call implied volatilities did not converge")

In [7]:
def merge_options(df_expiry):
    """
    merge calls and puts for a given expiry date and quote date using strike price as the key
    - this will produce a straddle layout and keep only strike prices that exist on both the call and put side
    """

    df_calls = df_expiry[df_expiry['option_type'] == 'call'].reset_index(drop=True)
    df_calls.rename(columns={'bid': 'call_bid', 'ask': 'call_ask', 'volume': 'call_volume', 'open_interest' : 'call_open_interest'}, inplace=True)
    del df_calls['option_type']

    df_puts = df_expiry[df_expiry['option_type'] == 'put'][['bid', 'ask', 'volume', 'open_interest', 'strike_price']].reset_index(drop=True)
    df_puts.rename(columns={'bid': 'put_bid', 'ask': 'put_ask', 'volume': 'put_volume', 'open_interest' : 'put_open_interest'}, inplace=True)

    df_straddle = pd.merge(df_calls, df_puts, on='strike_price', how='inner')
    return df_straddle


In [185]:
def create_base_record(quote_date, expiry_date, underlying_price, atm_price, atm_iv):
    """Generate the non repeating part of an option record"""
    base_record = {}
    base_record['quote_date'] = quote_date
    base_record['expiry_date'] = expiry_date
    base_record['underlying_price'] = underlying_price
    base_record['atm_price'] = round(atm_price, 2)
    base_record['atm_iv'] = atm_iv
    base_record['strike_prices'] = []
    base_record['sampling_key'] = random.random()
    return base_record


In [164]:
def calc_put_moneyness(atm_price, atm_iv, strike_price, days_to_expiry):
    """
    calculate the probability that a put option will close in the money on expiry date
    Note that the probability of a call option with the same strike price closing in the money
    will be 1 - the probability of the put optionclosing in the money    
    """
    iv_to_expiry = atm_iv * math.sqrt(days_to_expiry / 365)
    zscore = math.log(strike_price /  atm_price) / iv_to_expiry
    norm_cdf = scipy.stats.norm.cdf(zscore)
    return round(norm_cdf, 4)
    

In [181]:
def create_strike_record(row):
    """Generate a strike price record"""
    strike_record = {}
    strike_record['strike_price'] = row['strike_price']
       
    #call attributes
    strike_record['call_bid'] = row['call_bid']
    strike_record['call_ask'] = row['call_ask']
    strike_record['call_volume'] = row['call_volume']
    strike_record['call_open_interest'] = row['call_open_interest']
    strike_record['call_moneyness'] = round(1 - row['put_moneyness'], 4)
    strike_record['call_iv'] = row['call_iv']
    
     #put attributes
    strike_record['put_bid'] = row['put_bid']
    strike_record['put_ask'] = row['put_ask']
    strike_record['put_volume'] = row['put_volume']
    strike_record['put_open_interest'] = row['put_open_interest']
    strike_record['put_moneyness'] = row['put_moneyness']
    strike_record['put_iv'] = row['put_iv']
        
    return strike_record
    

In [179]:
def process_expiry_date(df_expiry, interest_rate):
    """
    read and process options for a given expiry date
    """
    #transpose calls and puts into a straddle layout
    df_straddle = merge_options(df_expiry)

    #find nearest strike price
    target_index = abs(df_straddle['strike_price'] - df_straddle['underlying_price']).idxmin()
    df_strike = df_straddle.loc[target_index]

    #get option values at target strike price
    quote_date = df_strike['quote_date'].strftime('%Y-%m-%d')
    expiry_date = df_strike['expiry_date'].strftime('%Y-%m-%d')
    underlying_price = df_strike['underlying_price']
    strike_price = df_strike['strike_price']
    days_to_expiry = (df_strike['expiry_date'] -  df_strike['quote_date']).days 
    call_price = (df_strike['call_bid'] + df_strike['call_ask']) / 2
    put_price = (df_strike['put_bid'] + df_strike['put_ask']) / 2

    #find adjusted underlying price and Implied Volatility where call and put implied volatility are the same
    atm_price, atm_iv = center_underlying_price(underlying_price, strike_price, interest_rate, 
        days_to_expiry, call_price, put_price)    
  
    #calculate put moneyness and exclude options that are more that 2 stard deviations in or out of the money
    df_straddle['put_moneyness'] = df_straddle.apply(lambda x: calc_put_moneyness(atm_price, atm_iv, x['strike_price'], days_to_expiry), axis=1)
    df_straddle = df_straddle[(df_straddle['put_moneyness'] > .05) & (df_straddle['put_moneyness'] < .95)].reset_index(drop=True)

    #calculate call implied volatilities using atm_price as the current price
    df_straddle['call_iv'] = df_straddle.apply(lambda x: calc_call_iv(atm_price, x['strike_price'], \
        interest_rate, days_to_expiry, (x['call_bid'] + x['call_ask']) / 2), axis=1)
  
    #calculate put implied volatilities using atm_price as the current price
    df_straddle['put_iv'] = df_straddle.apply(lambda x: calc_put_iv(atm_price, x['strike_price'], \
        interest_rate, days_to_expiry, (x['put_bid'] + x['put_ask']) / 2), axis=1)
  
    #create base record
    base_record = create_base_record(quote_date, expiry_date, underlying_price, atm_price, atm_iv)

    #add strike price records
    for index, row in df_straddle.iterrows():
        strike_record =  create_strike_record(row)
        base_record['strike_prices'].append(strike_record)
 
    return base_record

    

In [187]:
base_record = process_expiry_date(df_expiry, interest_rate)
base_record['strike_prices'][:5]

[{'strike_price': 315.0,
  'call_bid': 8.16,
  'call_ask': 8.23,
  'call_volume': 245,
  'call_open_interest': 4003,
  'call_moneyness': 0.94,
  'call_iv': 0.1545,
  'put_bid': 0.42,
  'put_ask': 0.43,
  'put_volume': 25710,
  'put_open_interest': 9249,
  'put_moneyness': 0.06,
  'put_iv': 0.1535},
 {'strike_price': 316.0,
  'call_bid': 7.26,
  'call_ask': 7.3,
  'call_volume': 164,
  'call_open_interest': 2500,
  'call_moneyness': 0.9115,
  'call_iv': 0.1489,
  'put_bid': 0.51,
  'put_ask': 0.52,
  'put_volume': 20226,
  'put_open_interest': 7427,
  'put_moneyness': 0.0885,
  'put_iv': 0.1483},
 {'strike_price': 317.0,
  'call_bid': 6.37,
  'call_ask': 6.42,
  'call_volume': 198,
  'call_open_interest': 2175,
  'call_moneyness': 0.874,
  'call_iv': 0.1437,
  'put_bid': 0.62,
  'put_ask': 0.63,
  'put_volume': 3336,
  'put_open_interest': 2850,
  'put_moneyness': 0.126,
  'put_iv': 0.1431},
 {'strike_price': 317.5,
  'call_bid': 5.92,
  'call_ask': 6.01,
  'call_volume': 27,
  'call_op

In [11]:
def process_quote_date(df_daily, quote_date):
    """
    process options for a given quote date
    """
    #get fed funds rate that was effective on the supplied quote date
    interest_rate = get_interest_rate(quote_date)
    
    #get list of expiry dates
    expiry_dates = df_daily['expiry_date'].unique()
    
    #process each expiry date
    for expiry_date in expiry_dates:
        df_expiry = df_daily[df_daily['expiry_date'] == expiry_date].reset_index(drop=True)
        process_expiry_date(df_expiry, interest_rate)

    

In [12]:
def main(file_path):
    """
    file path is path to options input file
    example: gs://expiry-week-data/options/options/SPY_2020.csv
    """
    #load file into a dataframe
    input_columns = [0,1,5,6,7,8,10,11,12,13]

    column_names = ['underlying_symbol', 'underlying_price', 'option_type', 'expiry_date', 'quote_date',
        'strike_price', 'bid', 'ask', 'volume', 'open_interest']

    df_options = pd.read_csv(file_path, usecols=input_columns, names=column_names, header=0, parse_dates=[3,4])
    
    #get unique list of quote dates
    quote_dates = df['quote_date'].unique()
    
    #process each quote date
    for quote_date in quote_dates:
        df_daily = df[df['quote_date'] == quote_date]
        process_quote_date(df_daily, quote_date)
    
    
    

## Code Exploration Section

In [13]:
#load sample options file from cloud storagae
 #load file into a dataframe
file_path  = 'gs://expiry-week-data/options/sample_20200103.csv'
input_columns = [0,1,5,6,7,8,10,11,12,13]

column_names = ['underlying_symbol', 'underlying_price', 'option_type', 'expiry_date', 'quote_date',
    'strike_price', 'bid', 'ask', 'volume', 'open_interest']

df_options = pd.read_csv(file_path, usecols=input_columns, names=column_names, header=0, parse_dates=[3,4])

#get set of options for one expiry date for the sample quote date

df_expiry = df_options[df_options['expiry_date'] == '2020-01-10'].reset_index(drop=True)
df_expiry.head()

interest_rate = get_interest_rate(df_expiry['quote_date'][0])
print(interest_rate)




1.55
